#HIDS-506 Final Project: Exploratory Data Analysis and Data Cleaning (triage.csv and edstays.csv)

## Packages



We will start by loading some of the packages that will help us 
organize and visualize the data. Other packages will be loaded as necessary.

In [ ]:
#Import packages
from collections import Counter
import pandas as pd
import numpy as np
import collections, numpy
import mlxtend
import matplotlib 
from matplotlib import pyplot 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
matplotlib.rcParams['figure.figsize'] = (10, 10)


from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc

## Dataset

**Research Question:** Can machine learning (ML) provide a data-driven alternative to current emergency care triage and produce an accurate patient acuity score based on minimal patient information upon admission to the Emergency Department (ED), thus reducing human-biased medical decisions and preserving clinician time, attention, and ED workflow? 

Triage Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **Temperature:** Triage vital sign; first temperature recording of patient upon ED arrival (Fahrenheit)
* **Heart Rate:** Triage vital sign; first record of patient heart rate upon arrival to ED (bpm)
* **Respiration Rate:** Triage vital sign; first record of patient respiration rate upon arrival to ED (breaths per minute)
* **Oxygen Saturation:** Triage vital sign; first record of patient oxygen saturation level - a measure of how much hemoglobin is currently bound to oxygen compared to how much hemoglobin remains unbound - upon arrival to ED (%)
* **Systolic Blood Pressure:** Triage vital sign; first record of patient systolic blood pressure upon arrival to ED (mmHg)
* **Diastolic Blood Pressure:** Triage vital sign; first record of patient diastolic blood pressure upon arrival to ED (mmHg)
* **Pain:** Triage vital sign; first record of patient reported pain upon arrival to ED (scale 0-10)
* **Chief Complaint:** Patient reported reason for admission to ED 
* **Outcome/Acuity:** Multi-class variable (1-5 patient condition severity rating upon arrival to the ED)

ED-Stays Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **In-Time:** Arrival time of patient to ED (datetime)
* **Out-Time:** Discharge/transfer time out of ED (datetime)
* **Gender:** Male or Female
* **Race:** White, black/african american, hispanic/latinom asian, portugese, south american, american indian/alaska native, native hawaiian or other pacific islander 
* **Arrival Transport:** Walk-in, ambulance, unknown, other, helicopter 
* **Disposition:** Patient discharge/transfer location



Exploring the datasets:

#### Triage Dataset

In [ ]:
#from google.colab import 
#drivedrive.mount('/content/drive')

In [ ]:
#mounting google drive to access files 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#reading in triage csv file
triage_file = "/content/drive/MyDrive/HIDS_506_FinalProject/data/mimic-iv-ed-2.2/ed/triage.csv.gz"

In [ ]:
#assigning triage csv file to dataframe
triage_df = pd.read_csv(triage_file)

In [ ]:
#visualizing first rows of triage dataframe
triage_df.head()

subject_id   stay_id  temperature  heartrate  resprate  o2sat    sbp   dbp  \
0    10000032  32952584         97.8       87.0      14.0   97.0   71.0  43.0   
1    10000032  33258284         98.4       70.0      16.0   97.0  106.0  63.0   
2    10000032  35968195         99.4      105.0      18.0   96.0  106.0  57.0   
3    10000032  38112554         98.9       88.0      18.0   97.0  116.0  88.0   
4    10000032  39399961         98.7       77.0      16.0   98.0   96.0  50.0   

  pain  acuity                            chiefcomplaint  
0    7     2.0                               Hypotension  
1    0     3.0            Abd pain, Abdominal distention  
2   10     3.0                           n/v/d, Abd pain  
3   10     3.0                      Abdominal distention  
4   13     2.0  Abdominal distention, Abd pain, LETHAGIC

In [ ]:
#visualizing the last few rows of triage dataframe
triage_df.tail()

subject_id   stay_id  temperature  heartrate  resprate  o2sat    sbp  \
425082    19999784  37972930         98.0       91.0      16.0   99.0  148.0   
425083    19999828  30712109         98.1       83.0      18.0  100.0  107.0   
425084    19999828  32917002         96.6      112.0      18.0  100.0  110.0   
425085    19999914  32002659         99.5       81.0      10.0  100.0   93.0   
425086    19999987  34731548          NaN        NaN       NaN    NaN    NaN   

         dbp pain  acuity                        chiefcomplaint  
425082  90.0    5     2.0                          Abnormal MRI  
425083  75.0    8     2.0                  Abd pain, Wound eval  
425084  82.0    4     2.0                  Abd pain, Wound eval  
425085  55.0    0     2.0  Altered mental status, Substance use  
425086   NaN  NaN     NaN                            HEAD BLEED

In [ ]:
#shape of triage dataframe: 11 features (including 1 outcome feature column 'acuity'), 425087 observations 
triage_df.shape

(425087, 11)

Handle missing values

In [ ]:
#return count of missing values per feature column in traige dataframe
triage_df.isnull().sum()

subject_id            0
stay_id               0
temperature       23415
heartrate         17090
resprate          20353
o2sat             20596
sbp               18291
dbp               19091
pain              12933
acuity             6987
chiefcomplaint       23
dtype: int64

Replacing other missing values by median of the respective column values

In [ ]:
#fills NA values with the median of a relevant feature excluding pain and acuity scores
# pain will be excluded from filling NA because we will only keep recorded pain scores of 0-10 for standardization
# NAs for acuity scores will not be filled because we will only keep patient observations that resulted in being triaged (or assigned our intended outcome variable)

triage_df['temperature'].fillna(triage_df['temperature'].median(), inplace=True) 
triage_df['heartrate'].fillna(triage_df['heartrate'].median(), inplace=True)
triage_df['resprate'].fillna(triage_df['resprate'].median(), inplace=True)
triage_df['o2sat'].fillna(triage_df['o2sat'].median(), inplace=True)
triage_df['sbp'].fillna(triage_df['sbp'].median(), inplace=True)
triage_df['dbp'].fillna(triage_df['dbp'].median(), inplace=True)



In [ ]:
triage_df.isnull().sum()

subject_id            0
stay_id               0
temperature           0
heartrate             0
resprate              0
o2sat                 0
sbp                   0
dbp                   0
pain              12933
acuity             6987
chiefcomplaint       23
dtype: int64

In [ ]:
#visualizing count of specific recorded pain scores, will keep standard patient reported pain scores from 0 (no pain) to 10 (most pain) for consistency 
# pain scores recorded with values '0'-'10' were also the most common recorded values in the pain column 
triage_df['pain'].value_counts()

0                 140719
8                  41730
10                 40914
7                  31423
5                  29210
                   ...  
almost nothing         1
"up there"             1
achey                  1
. 10                   1
2-10                   1
Name: pain, Length: 847, dtype: int64

In [ ]:
Counter(triage_df['pain'])

In [ ]:
triage_df = triage_df.loc[triage_df['pain'].isin(['0','1','2','3','4','5','6','7','8','9','10'])]

In [ ]:
Counter(triage_df['pain'])

Counter({'7': 31423,
         '0': 140719,
         '10': 40914,
         '5': 29210,
         '8': 41730,
         '1': 5903,
         '3': 16115,
         '6': 25206,
         '2': 13903,
         '4': 19721,
         '9': 20210})

In [ ]:
# reduced observations to 385054
triage_df.shape

(385054, 11)

In [ ]:
# there are now no missing values in the pain feature column
triage_df.isnull().sum()

subject_id          0
stay_id             0
temperature         0
heartrate           0
resprate            0
o2sat               0
sbp                 0
dbp                 0
pain                0
acuity            174
chiefcomplaint     12
dtype: int64

In [ ]:
#dropping all rows of patients who do not have an assigned acuity score 
triage_df = triage_df.dropna(subset=['acuity'])
triage_df.shape
# 11 features with 384880 observations 

(384880, 11)

In [ ]:
triage_df.head()

subject_id   stay_id  temperature  heartrate  resprate  o2sat    sbp   dbp  \
0    10000032  32952584         97.8       87.0      14.0   97.0   71.0  43.0   
1    10000032  33258284         98.4       70.0      16.0   97.0  106.0  63.0   
2    10000032  35968195         99.4      105.0      18.0   96.0  106.0  57.0   
3    10000032  38112554         98.9       88.0      18.0   97.0  116.0  88.0   
5    10000084  35203156         97.5       78.0      16.0  100.0  114.0  71.0   

  pain  acuity                  chiefcomplaint  
0    7     2.0                     Hypotension  
1    0     3.0  Abd pain, Abdominal distention  
2   10     3.0                 n/v/d, Abd pain  
3   10     3.0            Abdominal distention  
5    0     2.0       Confusion, Hallucinations

In [ ]:
triage_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384880 entries, 0 to 425085
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   subject_id      384880 non-null  int64  
 1   stay_id         384880 non-null  int64  
 2   temperature     384880 non-null  float64
 3   heartrate       384880 non-null  float64
 4   resprate        384880 non-null  float64
 5   o2sat           384880 non-null  float64
 6   sbp             384880 non-null  float64
 7   dbp             384880 non-null  float64
 8   pain            384880 non-null  object 
 9   acuity          384880 non-null  float64
 10  chiefcomplaint  384868 non-null  object 
dtypes: float64(7), int64(2), object(2)
memory usage: 35.2+ MB


In [ ]:
#changing pain data type from object to int64
triage_df['pain'] = triage_df['pain'].astype('int64')

In [ ]:
triage_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384880 entries, 0 to 425085
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   subject_id      384880 non-null  int64  
 1   stay_id         384880 non-null  int64  
 2   temperature     384880 non-null  float64
 3   heartrate       384880 non-null  float64
 4   resprate        384880 non-null  float64
 5   o2sat           384880 non-null  float64
 6   sbp             384880 non-null  float64
 7   dbp             384880 non-null  float64
 8   pain            384880 non-null  int64  
 9   acuity          384880 non-null  float64
 10  chiefcomplaint  384868 non-null  object 
dtypes: float64(7), int64(3), object(1)
memory usage: 35.2+ MB


In [ ]:
#coverting categorical features
#chief complaint feature will need to be one-hot-encoded 
triage_df['chiefcomplaint'].value_counts()

Chest pain                        10808
Abd pain                          10416
Dyspnea                            5418
SI                                 4745
ABD PAIN                           4664
                                  ...  
L FA PAIN                             1
Abnormal labs, R Shoulder pain        1
Transfer, VERTREBRAL DISECTION        1
Weakness, Myalgia                     1
? SEIZURE TODAY                       1
Name: chiefcomplaint, Length: 55431, dtype: int64

### ED-Stays Dataset

In [ ]:
# defining edstays file path
edstays_file = "/content/drive/MyDrive/HIDS_506_FinalProject/data/mimic-iv-ed-2.2/ed/edstays.csv.gz"

# reading edstays file into dataframe
edstays_df = pd.read_csv(edstays_file)

In [ ]:
# visualizing the first few rows of edstays.csv
edstays_df.head()

subject_id     hadm_id   stay_id               intime              outtime  \
0    10000032  22595853.0  33258284  2180-05-06 19:17:00  2180-05-06 23:30:00   
1    10000032  22841357.0  38112554  2180-06-26 15:54:00  2180-06-26 21:31:00   
2    10000032  25742920.0  35968195  2180-08-05 20:58:00  2180-08-06 01:44:00   
3    10000032  29079034.0  32952584  2180-07-22 16:24:00  2180-07-23 05:54:00   
4    10000032  29079034.0  39399961  2180-07-23 05:54:00  2180-07-23 14:00:00   

  gender   race arrival_transport disposition  
0      F  WHITE         AMBULANCE    ADMITTED  
1      F  WHITE         AMBULANCE    ADMITTED  
2      F  WHITE         AMBULANCE    ADMITTED  
3      F  WHITE         AMBULANCE        HOME  
4      F  WHITE         AMBULANCE    ADMITTED

In [ ]:
edstays_df.isnull().sum()

subject_id                0
hadm_id              222071
stay_id                   0
intime                    0
outtime                   0
gender                    0
race                      0
arrival_transport         0
disposition               0
dtype: int64

In [ ]:
# gender: M/F 
# categorical variable will be one-hot-encoded 
edstays_df['gender'].value_counts()

F    229898
M    195189
Name: gender, dtype: int64

In [ ]:
# assign Female = 0 and Male = 1
#edstays_df['gender'].replace('F', 0, inplace=True)
#edstays_df['gender'].replace('M', 1, inplace=True)
#edstays_df['gender'].value_counts()

In [ ]:
# value counts of race column
edstays_df['race'].value_counts()

WHITE                                        228123
BLACK/AFRICAN AMERICAN                        76798
OTHER                                         20752
HISPANIC/LATINO - PUERTO RICAN                14036
WHITE - OTHER EUROPEAN                         8992
HISPANIC/LATINO - DOMINICAN                    8330
BLACK/CAPE VERDEAN                             7638
ASIAN - CHINESE                                7348
ASIAN                                          7294
UNKNOWN                                        7083
WHITE - RUSSIAN                                6091
BLACK/AFRICAN                                  4887
BLACK/CARIBBEAN ISLAND                         3675
HISPANIC OR LATINO                             3141
HISPANIC/LATINO - GUATEMALAN                   2356
ASIAN - ASIAN INDIAN                           1567
ASIAN - SOUTH EAST ASIAN                       1533
HISPANIC/LATINO - SALVADORAN                   1497
WHITE - BRAZILIAN                              1484
PORTUGUESE  

In [ ]:
# cleaning race column by re-coding subsets into generalized 'race' groups 
edstays_df['race'].replace('WHITE - OTHER EUROPEAN', 'WHITE', inplace=True)
edstays_df['race'].replace('WHITE - RUSSIAN', 'WHITE', inplace=True)
edstays_df['race'].replace('WHITE - BRAZILIAN', 'WHITE', inplace=True)
edstays_df['race'].replace('WHITE - EASTERN EUROPEAN', 'WHITE', inplace=True)

edstays_df['race'].replace('BLACK/CAPE VERDEAN', 'BLACK/AFRICAN AMERICAN', inplace=True)
edstays_df['race'].replace('BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', inplace=True)
edstays_df['race'].replace('BLACK/CARIBBEAN ISLAND', 'BLACK/AFRICAN AMERICAN', inplace=True)
edstays_df['race'].replace('BLACK/CARIBBEAN ISLAND', 'BLACK/AFRICAN AMERICAN', inplace=True)

edstays_df['race'].replace('HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - DOMINICAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC OR LATINO', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - SALVADORAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - COLUMBIAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - HONDURAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO', inplace=True)
edstays_df['race'].replace('HISPANIC/LATINO - CENTRAL AMERICAN', 'HISPANIC/LATINO', inplace=True)

edstays_df['race'].replace('ASIAN - CHINESE', 'ASIAN', inplace=True)
edstays_df['race'].replace('ASIAN - ASIAN INDIAN', 'ASIAN', inplace=True)
edstays_df['race'].replace('ASIAN - SOUTH EAST ASIAN', 'ASIAN', inplace=True)
edstays_df['race'].replace('ASIAN - KOREAN', 'ASIAN', inplace=True)

edstays_df['race'].replace('MULTIPLE RACE/ETHNICITY', 'OTHER/UNKNOWN', inplace=True)
edstays_df['race'].replace('OTHER', 'OTHER/UNKNOWN', inplace=True)
edstays_df['race'].replace('UNKNOWN', 'OTHER/UNKNOWN', inplace=True)
edstays_df['race'].replace('UNABLE TO OBTAIN', 'OTHER/UNKNOWN', inplace=True)
edstays_df['race'].replace('PATIENT DECLINED TO ANSWER', 'OTHER/UNKNOWN', inplace=True)


In [ ]:
# re-visualizing race value counts
edstays_df['race'].value_counts()

WHITE                                        246002
BLACK/AFRICAN AMERICAN                        92998
HISPANIC/LATINO                               34511
OTHER/UNKNOWN                                 28984
ASIAN                                         18528
PORTUGUESE                                     1457
SOUTH AMERICAN                                 1063
AMERICAN INDIAN/ALASKA NATIVE                  1037
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER       507
Name: race, dtype: int64

In [ ]:
# 9 features, 394646 observations
edstays_df.shape

(425087, 9)

In [ ]:
# recoding races to numerical categorical values to be one-hot-encoded
# white = 0, black/african american = 1, hispanic/latino = 2, asian = 3, south american = 4, american indian = 5, native hawaiian/pacific islander = 6
#edstays_df['race'].replace('WHITE', 0, inplace=True)
#edstays_df['race'].replace('BLACK/AFRICAN AMERICAN', 1, inplace=True)
#edstays_df['race'].replace('HISPANIC/LATINO', 2, inplace=True)
#edstays_df['race'].replace('ASIAN', 3, inplace=True)
#edstays_df['race'].replace('SOUTH AMERICAN', 4, inplace=True)
#edstays_df['race'].replace('AMERICAN INDIAN/ALASKA NATIVE', 5, inplace=True)
#edstays_df['race'].replace('NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 6, inplace=True)


In [ ]:
# re-visualizing first few rows of edstays 
edstays_df.head()

subject_id     hadm_id   stay_id               intime              outtime  \
0    10000032  22595853.0  33258284  2180-05-06 19:17:00  2180-05-06 23:30:00   
1    10000032  22841357.0  38112554  2180-06-26 15:54:00  2180-06-26 21:31:00   
2    10000032  25742920.0  35968195  2180-08-05 20:58:00  2180-08-06 01:44:00   
3    10000032  29079034.0  32952584  2180-07-22 16:24:00  2180-07-23 05:54:00   
4    10000032  29079034.0  39399961  2180-07-23 05:54:00  2180-07-23 14:00:00   

  gender   race arrival_transport disposition  
0      F  WHITE         AMBULANCE    ADMITTED  
1      F  WHITE         AMBULANCE    ADMITTED  
2      F  WHITE         AMBULANCE    ADMITTED  
3      F  WHITE         AMBULANCE        HOME  
4      F  WHITE         AMBULANCE    ADMITTED

In [ ]:
edstays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425087 entries, 0 to 425086
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   subject_id         425087 non-null  int64  
 1   hadm_id            203016 non-null  float64
 2   stay_id            425087 non-null  int64  
 3   intime             425087 non-null  object 
 4   outtime            425087 non-null  object 
 5   gender             425087 non-null  object 
 6   race               425087 non-null  object 
 7   arrival_transport  425087 non-null  object 
 8   disposition        425087 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 29.2+ MB


In [ ]:
#converting in and out times from ed to datetime objects
edstays_df['intime'] = pd.to_datetime(edstays_df['intime'])
edstays_df['outtime'] = pd.to_datetime(edstays_df['outtime'])

In [ ]:
edstays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425087 entries, 0 to 425086
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   subject_id         425087 non-null  int64         
 1   hadm_id            203016 non-null  float64       
 2   stay_id            425087 non-null  int64         
 3   intime             425087 non-null  datetime64[ns]
 4   outtime            425087 non-null  datetime64[ns]
 5   gender             425087 non-null  object        
 6   race               425087 non-null  object        
 7   arrival_transport  425087 non-null  object        
 8   disposition        425087 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 29.2+ MB


In [ ]:
# value counts of arrival transport
edstays_df['arrival_transport'].value_counts()

WALK IN       251849
AMBULANCE     155752
UNKNOWN        15352
OTHER           1266
HELICOPTER       868
Name: arrival_transport, dtype: int64

In [ ]:
#edstays_df['arrival_transport'].replace('WALK IN', 0, inplace=True)
#edstays_df['arrival_transport'].replace('AMBULANCE', 1, inplace=True)
#edstays_df['arrival_transport'].replace('HELICOPTER', 2, inplace=True)
#edstays_df['arrival_transport'].replace('UNKNOWN', 3, inplace=True)
#edstays_df['arrival_transport'].replace('OTHER', 4, inplace=True)

In [ ]:
# value counts of disposition (patient discharge/transfer location)
edstays_df['disposition'].value_counts()

HOME                           241632
ADMITTED                       158010
TRANSFER                         7025
LEFT WITHOUT BEING SEEN          6155
ELOPED                           5710
OTHER                            4297
LEFT AGAINST MEDICAL ADVICE      1881
EXPIRED                           377
Name: disposition, dtype: int64

In [ ]:
edstays_df.isnull().sum()
#hadm_id will be dropped for analysis

subject_id                0
hadm_id              222071
stay_id                   0
intime                    0
outtime                   0
gender                    0
race                      0
arrival_transport         0
disposition               0
dtype: int64

In [ ]:
# saving cleaned dataframes to csv 
triage_df.to_csv('/content/drive/MyDrive/HIDS_506_FinalProject/data/triage_clean.csv')
edstays_df.to_csv('/content/drive/MyDrive/HIDS_506_FinalProject/data/edstays_clean.csv')